In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import *
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, accuracy_score

In [ ]:

# Define data augmentation parameters
data_augmentation = ImageDataGenerator(
    rotation_range=20,   # Rotate the image by up to 20 degrees
    width_shift_range=0.1,   # Shift the image horizontally by up to 10% of the image width
    height_shift_range=0.1,   # Shift the image vertically by up to 10% of the image height
    shear_range=0.2,   # Shear the image by up to 20%
    zoom_range=0.2,   # Zoom the image by up to 20%
    horizontal_flip=True,   # Flip the image horizontally
    vertical_flip=True,
    fill_mode='nearest'   # Fill any empty pixels with the nearest pixel value
)
t_data_augmentation = ImageDataGenerator(
    fill_mode='nearest'   # Fill any empty pixels with the nearest pixel value
)

batch_size=32
train_generator = data_augmentation.flow_from_directory(
    '/content/train',
    classes=['Abnormal', 'Normal'],  # exclude .ipynb_checkpoints directory
    target_size=(336, 448),
    batch_size=32,
    class_mode='binary',
    shuffle=True)


validation_generator = data_augmentation.flow_from_directory('/content/validation',
                                             classes=['Abnormal', 'Normal'],
                                             target_size=(336, 448),
                                             batch_size=32,
                                             class_mode='binary',
                                             shuffle=True)

test_generator = t_data_augmentation.flow_from_directory('/content/test',
                                              classes=['Abnormal', 'Normal'],
                                             target_size=(336, 448),
                                             batch_size=32,
                                             class_mode='binary',
                                             shuffle=True)

class_names = train_generator.class_indices
print(class_names)

Found 268 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
{'Abnormal': 0, 'Normal': 1}


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Flatten, Dense

# Define the CNN model
model1 = Sequential()

# Add the input layer
model1.add(Conv2D(32, (3, 3), padding='same', input_shape=(336,448, 3)))

# Add the first max pooling layer
model1.add(MaxPooling2D(pool_size=(2, 2)))

# Add the first batch normalization layer
model1.add(BatchNormalization())

# Add the first ReLU activation layer
model1.add(Activation('relu'))

# Add the second max pooling layer
model1.add(MaxPooling2D(pool_size=(2, 2)))

# Add the second batch normalization layer
model1.add(BatchNormalization())

# Add the second ReLU activation layer
model1.add(Activation('relu'))

# Add the bottleneck layer
model1.add(Conv2D(64, (1,1), padding='same'))
model1.add(Activation('relu'))
model1.add(BatchNormalization())
model1.add(Conv2D(64, (3, 3), padding='same'))
model1.add(Activation('relu'))
model1.add(BatchNormalization())
model1.add(Conv2D(32, (1,1), padding='same'))
model1.add(Activation('relu'))
model1.add(BatchNormalization())
# Add the third ReLU activation layer
model1.add(Activation('relu'))

# Add the flattening layer
model1.add(Flatten())

# Add the fully connected layer
model1.add(Dense(128))

# Add the third batch normalization layer
model1.add(BatchNormalization())

# Add the fourth ReLU activation layer
model1.add(Activation('relu'))

# Add the output layer with softmax activation function for classification
model1.add(Dense(2, activation='softmax'))

# Compile the model1 with categorical cross-entropy loss function and Adam optimizer


In [ ]:
model1.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics = ['accuracy'])
batch_size = 16
n_splits = 5


# Define the cross-validation folds
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Loop through each fold
fold_scores = []
for fold, (train_indices, val_indices) in enumerate(kfold.split(train_generator)):
    print(f"Fold {fold+1}:")
    # Train the model on the training set and validate on the validation set
    history=model1.fit(
        train_generator,   # Use the training data generator
        steps_per_epoch=train_generator.n // train_generator.batch_size,   # Number of steps per epoch
        epochs=20,  # Train for 20 epochs
        validation_data=validation_generator,  # Validation data generator
        validation_steps=validation_generator.n // validation_generator.batch_size   # Number of validation steps
    )
    # Evaluate the model on the validation set
    scores = model1.evaluate(validation_generator)
    fold_scores.append(scores[1])
    print(scores)

# Calculate the average accuracy across all folds
mean_accuracy = np.mean(fold_scores)
print(f"Mean validation accuracy: {mean_accuracy}")


Fold 1:
Epoch 1/20
8/8 [==============================] - 81s 9s/step - loss: 0.4173 - accuracy: 0.8305 - val_loss: 4.6968 - val_accuracy: 0.4375
Epoch 2/20
8/8 [==============================] - 78s 10s/step - loss: 0.2610 - accuracy: 0.9153 - val_loss: 3.4426 - val_accuracy: 0.2500
Epoch 3/20
8/8 [==============================] - 84s 10s/step - loss: 0.2290 - accuracy: 0.9322 - val_loss: 2.5956 - val_accuracy: 0.4688
Epoch 4/20
8/8 [==============================] - 76s 9s/step - loss: 0.1473 - accuracy: 0.9492 - val_loss: 2.9217 - val_accuracy: 0.2500
Epoch 5/20
8/8 [==============================] - 82s 10s/step - loss: 0.1236 - accuracy: 0.9576 - val_loss: 2.8713 - val_accuracy: 0.0625
Epoch 6/20
8/8 [==============================] - 81s 10s/step - loss: 0.1570 - accuracy: 0.9648 - val_loss: 2.9794 - val_accuracy: 0.4062
Epoch 7/20
8/8 [==============================] - 74s 9s/step - loss: 0.1565 - accuracy: 0.9619 - val_loss: 2.3278 - val_accuracy: 0.5000
Epoch 8/20
8/8 [======

In [ ]:
model1.save('model.h5')

In [ ]:
# predict the output for the test data
test_preds = model1.predict(test_generator)
# convert the predictions to 0 or 1
test_preds = np.round(test_preds)
# generate a classification report and find the accuracy
test_labels = test_generator.classes
class_names = list(test_generator.class_indices.keys())
######
test_labels = test_generator.classes
# print(test_labels)

2/2 [==============================] - 4s 515ms/step


In [ ]:
import numpy as np
indices=[]
# Create a 2D numpy ndarray with ones and zeroes
arr = test_preds
# Find the indices of the ones in the array
for i in arr:
  indices.extend(list(np.where(i== 1)[0]))
  # np.append(indices,axis=0)

# Print the resulting 1D array
indices=np.array(indices)

# print(indices)
# print(test_labels)
print(class_names)
test_preds=np.array(indices)

['Abnormal', 'Normal']


In [ ]:
import numpy as np
indices=[]
# Create a 2D numpy ndarray with ones and zeroes
arr = test_preds
# Find the indices of the ones in the array
for i in arr:
  indices.extend(list(np.where(i== 1)[0]))
  # np.append(indices,axis=0)

# Print the resulting 1D array
indices=np.array(indices)
# test_labels
# print(indices)
#getting the names of classes
print(class_names)
test_preds=np.array(indices)
import h5py
from sklearn.metrics import classification_report, accuracy_score, precision_score,recall_score
accuracy= accuracy_score(test_labels, test_preds)
precision= precision_score(test_labels, test_preds)
recall= recall_score(test_labels, test_preds)
print("Accuracy :",accuracy)
print("Precision :",precision)
print("Recall",recall)
#saving a model
model1.save('model.h5')

['Abnormal', 'Normal']
Accuracy : 0.972
Precision : 0.971
Recall 0.964


In [ ]:
from sklearn.metrics import classification_report
classifcation=classification_report(y_pred,y_test)
print(classification)

 precision    recall  f1-score   support

    Abnormal       0.98      1.00      0.99       102
      Normal         0.85      0.90      0.87         20

    accuracy                                  0.97       122
   macro avg       0.89      0.90      0.90       122
weighted avg     0.97      0.96      0.96       122

